In [73]:
import numpy as np
import pandas as pd
import random
from gensim.models import word2vec
import matplotlib.pyplot as plt
import operator

In [5]:
NUM_PROGRAM = 8
cut_programs = np.load('newCut/cut_Programs.npy')
cut_questions = np.load('newCut/cut_Questions.npy')

In [6]:
def add_word_dict(w, word_dict):
    if not w in word_dict:
        word_dict[w] = 1
    else:
        word_dict[w] += 1

def buildVocDict(programs, questions, VOC_START, VOC_SIZE):
    word_dict = dict()
    
    for program in programs:
        for lines in program:
            for line in lines:
                for w in line:
                    add_word_dict(w, word_dict)

    for question in questions:
        lines = question[0]
        for line in lines:
            for w in line:
                add_word_dict(w, word_dict)

        for i in range(1, 7):
            line = question[i]
            for w in line:
                add_word_dict(w, word_dict)
    ## Pop stopwords
    """for sw in stopwords:
        if sw in word_dict:
            word_dict.pop(sw) """  
    
    word_dict = sorted(word_dict.items(), key=operator.itemgetter(1), reverse=True)
    voc_dict = word_dict[VOC_START:VOC_START+VOC_SIZE]# VOC_START+VOC_SIZE
    return voc_dict

In [8]:
VOC_START = 20
VOC_SIZE = 30000
voc_dict = buildVocDict(cut_programs, cut_questions, VOC_START, VOC_SIZE)
np.save('newCut/voc_dict.npy', voc_dict)

In [12]:
## 還沒考慮 voc_dict 的 programs_questions 
programs_questions = []
num_lines = 0
with open('newCut/program_question.txt', 'r', encoding='utf-8') as inFile:
    for line in inFile:
        programs_questions.append(line.strip('\n'))
        num_lines += 1
        
        if num_lines % 400000 == 0:
            print ("Finished %d of lines" % num_lines)

Finished 400000 of lines
Finished 800000 of lines
Finished 1200000 of lines
Finished 1600000 of lines
Finished 2000000 of lines
Finished 2400000 of lines


In [13]:
sentences = word2vec.Text8Corpus("newCut/program_question.txt")
word2vec_model = word2vec.Word2Vec(sentences, min_count=1)
word2vec_model.save("newCut/model.bin")

In [15]:
word2vec_model = word2vec.Word2Vec.load('newCut/model.bin')

In [20]:
doc_vec = []
for sentence in programs_questions:
    words = sentence.split(' ')[:-1]
    vec = np.zeros(shape=(100,), dtype=float)
    count = 0
    for w in words:
        try:
            vec += word2vec_model.wv[w]
            count += 1
        except KeyError:
            pass
    doc_vec.append(vec)

In [21]:
vec_programs = doc_vec[:2396906]
vec_questions = doc_vec[2396906:]

In [22]:
line_programs = programs_questions[:2396906]
line_questions = programs_questions[2396906:]

In [116]:
import random
NUM_TRAIN = 200000

In [254]:
def generate_training_data(NUM_TRAIN=10000):
    Xs, Ys = [], []
    data_num = 0
    while data_num < NUM_TRAIN:
        if data_num%50000 == 0:
            print("[%d/%d]" % (data_num, NUM_TRAIN))
        pos_or_neg = random.randint(0, 1)
        
        if pos_or_neg==1:
            line_id = random.randint(0, len(line_programs)-1)
            if line_id+1 < len(line_programs)-1:
                L1 = line_programs[line_id].split(' ')[:-1] #原句
                L2 = line_programs[line_id+1].split(' ')[:-1] #下句
                L3 = line_programs[line_id-1].split(' ')[:-1] #上句

                if L1 and L2 and L3:
                    if word2vec_model.n_similarity(L1, L2) > word2vec_model.n_similarity(L1, L3):
                        Xs.append([vec_programs[line_id], vec_programs[line_id+1]])
                        Ys.append(1)
                        data_num += 1
                    else:
                        Xs.append([vec_programs[line_id], vec_programs[line_id-1]])
                        Ys.append(1)
                        data_num += 1
                    """sim1, sim2 = 0, 0
                    for w1 in L1:
                        for w2 in L2:
                            sim1 += word2vec_model.wv.similarity(w1, w2)
                        for w3 in L3:
                            sim2 +=  word2vec_model.wv.similarity(w1, w3)
                    if sim1 > sim2:
                        Xs.append([vec_programs[line_id], vec_programs[line_id+1]])
                        Ys.append(sim1/(len(L1)*len(L2)))
                        data_num += 1
                    else:
                        Xs.append([vec_programs[line_id], vec_programs[line_id-1]])
                        Ys.append(sim2/(len(L1)*len(L3)))
                        data_num += 1"""
            
        else:
            line_id1 = random.randint(0, len(line_programs)-1)
            line_id2 = random.randint(0, len(line_programs)-1)
            
            L1 = line_programs[line_id1].split(' ')[:-1]
            L2 = line_programs[line_id2].split(' ')[:-1]
            
            if (line_id2 != line_id1+1) and (line_id2 != line_id1-1):
                sim = 0
                if L1 and L2: 
                    """for w1 in L1:
                        for w2 in L2:
                            sim +=  word2vec_model.wv.similarity(w1, w2)"""
                    Xs.append([vec_programs[line_id1], vec_programs[line_id2]])
                    Ys.append(-1)
                    #Ys.append(sim/(len(L1)*len(L2)))
                    data_num += 1
    
    return Xs, Ys

In [255]:
X, y = generate_training_data(NUM_TRAIN)

[0/50000]


In [263]:
X_data = []

for t in X:
    X_data.append(np.dot(t[0], t[1])*t[0]/np.linalg.norm(t[0]))

X_data = np.stack(X_data, axis=0)

In [264]:
X_data.shape

(50000, 100)

In [266]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_data, y, test_size=0.7, random_state=1)

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_valid_std = sc.transform(X_valid)

In [267]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_valid)
print("MSE: %f" % (mean_squared_error(y_valid, y_pred)))

MSE: 0.937015


In [270]:
q_test = []
for q_id in range(500):
    question = line_questions[q_id*7].split(' ')[:-1]
    
    q_vec = np.zeros(shape=(100,), dtype=float)
    for w in question:
        q_vec += word2vec_model[w]
        
    answer = 0
    sim_max = sys.float_info.min
    for i in range(1,7):
        ans = line_questions[q_id*7 + i].split(' ')[:-1]
        #result = word2vec_model.n_similarity(question, ans)
        ans_vec = np.zeros(shape=(100,), dtype=float)
        for w in ans:
            ans_vec += word2vec_model[w]
            
        test_vec = []
        test_vec.append(np.dot(q_vec, ans_vec)*q_vec/np.linalg.norm(q_vec))
        test_vec = np.stack(test_vec, axis=0)
        
        result = lr.predict(test_vec)
        
        if result > sim_max:
            answer = i-1
            sim_max = result
    
    q_test.append((q_id, answer))

In [250]:
question = line_questions[0*7].split(' ')[:-1]
    
q_vec = np.zeros(shape=(100,), dtype=float)
for w in question:
    q_vec += word2vec_model[w]

ans = line_questions[0*7 + 4].split(' ')[:-1]
#result = word2vec_model.n_similarity(question, ans)
ans_vec = np.zeros(shape=(100,), dtype=float)
for w in ans:
    ans_vec += word2vec_model[w] 

data = []

data.append(list(q_vec)+list(ans_vec))
data = np.stack(data, axis=0)

"""test = np.stack(test, axis=0)
pred = lr.predict(test)"""
pred = lr.predict(data)
pred

array([ 0.18930214])

In [273]:
q_test
df = pd.DataFrame(data=q_test, columns=['Id', 'Answer']).to_csv('Answer.csv', index=False)

In [75]:
questions = pd.read_csv('Question.csv')
questions

,Question,Option0,Option1,Option2,Option3,Option4,Option5
0,媽給你送錢包來啦 來 你看一下是不是這個\n對 就是這個 你在哪裡找到它的\n,你看 這是我新買的錢包,我的錢包不見了啦,以後上網咖的錢包在我身上,什麼有錢包場,早上你爸爸在車上找到的 一定是前天你放學的時候掉在車上了,我為什麼要給你們錢包
1,古人說三日不讀書 面目可憎 我覺得我最近可能臉色太難看了\n,所以想回復我昔日面貌姣好的樣子,是不是要定期來舉辦,各辦理一次才對,能夠督促所有的用人機關,在上次的節目討論中也有提到,超過九十分貝以上
2,你說我們做父母的最擔心的就是這個\n,我剛剛聽你媽說你要讀什麼科,其他老師又集體叛變,聽起來好好玩天啊,只是小孩自己的興趣不能得到發展 他們的心裡可能也會很悶喔,走到這裡就沒有路了耶,每一個科目像是國語數學都很優秀
3,是不是因為以前你的書包太重了 所以你想要發明這個\n,這個應該也是其中之一的原因,好神奇喔,它怎麼突然會這樣,那我去探索一下,牠會不會覺得很不開心 會生氣,我覺得很高興
4,那可不見得 有錢能使鬼推磨\n,好 這個就是我們今天比較級的用法,真的嗎 什麼禮物,有餅乾吃我也要吃,我知道隔壁班今天也有地理課 我陪你去借,什麼你說我是鬼,我表姊從國外帶回來的 送你一個
5,真的嗎\n當然是真的囉 如果你不相信的話 可以問姜老師啊\n,老師常常看了都很感動,你不要太在意同學們怎麼想,老師先給你們一個功課,就是因為這樣 你們才要更包容他更鼓勵他,是啊 這點老師也可以證明喔,沒有人是十全十美的
6,你今天是要介紹哪一位人物 讓我們認識呢\n是和戲劇有關喔 特別是默劇\n,我們不是當事人，不太方便回答相關問題,兩人得到了法國航空提供的巴黎來回機票,這個人我已經知道是誰啦 就是耿一偉老師嘛,墨西哥是自古以來就吃昆蟲的國家,兩人相繼離世 親朋好友皆非常感傷,善用資源聰明借款 早日實現買屋夢想
7,輝哥 全部都被你吃了 你很過分\n就兩顆 你還剩下這樣子 那我吃什麼\n沒有 因為你們一直在...,然後煮一煮就變成紅色的嗎,我就把它吃完了,現在要把那個木頭給粉碎掉,因為聽說以前是用燒的,會造成空氣污染,像他們現在這樣粉碎掉
8,大嬸啊 我不是說過了嗎 我們村子都是善男信女 老弱婦孺 怎麼會是練功的料呢\n,看來大家對男生女生應該有什麼樣子 還是有很刻板的印象喔,我們讀不讀書關你屁事啊 你管那麼多幹嘛,錯了 我們的武功是要教大家 以小博大 以柔克剛 男女老幼都可以學習的,所以做母親的 我們覺得非常了不起非常偉大,這次你就假裝沒看到,好 就交給你們
9,你只要拿得到滅火器 就代表你的逃生動線是安全的 你就算火滅不掉\n,防災達人接下來要突擊的,是位在桃園八德市的一家雜貨店,之前台語歌曲天后張秀卿,曾跟店主人共進晚餐,你還是可以跑出去,終於以超人的毅力


In [ ]:
ans = [(0, 4), (1, 0), (2, 3), (3, 0), (4, 4),
       (5, 4), (6, 2), (7, 1), (8, 2), (9, 4),
       (10, 2), (11, ), (12, 4), (13, 1), (10, ), ]